In [1]:
import docx
import re
import sys
from win32com import client as wc
import docx
import os

In [2]:
# 处理文件为docx模式
# step1 获得所有文件
# step2 将每一个文件另存为docs模式

#返回一个list类型的文件目录
def GetAllFiles(path,path2):
    '''
    parameter:path 为源路径,path2为修改的路径
    '''
    if os.path.isdir(path):
        c = os.listdir(path)
        c = [path+"\\"+i for i in c]
        t = [path2+"\\"+i.split('\\')[-1].split('.')[0]+".docx" for i in c]
        return c,t
        print("="*20,"完成路径转化","="*20)

#处理单个dir里面的文件
#word = wc.Dispatch('Word.Application')
def DocSavesDocx(_dir,_dir2):
    doc = word.Documents.Open(_dir)
    doc.SaveAs(_dir2,12,False,"",True,"",False,False,False)
    doc.Close()
    print("="*20,"另存为完毕","="*20)
# ord.Quit()

In [3]:
# 现在可以进行正则化匹配了
#step1 打开文件

# _dic = {}
# count = 0
def GetAnswer(_dir2):
    _file = docx.Document(_dir2)
    str = ""
    for i in _file.paragraphs:
        str +=i.text+"\n"
    
    # 解决脏数据()（）[] 【】
    
    str = str.replace('(','【')
    str = str.replace('（','【')
    str = str.replace(' ','')
    str = str.replace(')','】')
    str = str.replace('）','】')
    str = str.replace('\t','')
    str = str.replace('\xa0','')
    
    # 正则化匹配
    result = re.findall('([0-9]{1,2}).*?【([A-Da-d])】.*?\n',str)
    name = _dir2.split("\\")[-1].split(".")[0]
    if(len(result)>=20):
        _dic[name]=result
    else:
        print("===",name,"====>匹配失败!!!!",len(result))

### 805班级

In [4]:
# 主程序
path = "E:\\qq_document\\2181778692\\FileRecv\\805复习1\\805复习1"
path2 = "E:\\qq_document\\2181778692\\FileRecv\\805复习1\\副本"
_dirlist,_dir2list = GetAllFiles(path,path2)

### 804班

In [12]:
path="E:\\qq_document\\2181778692\\FileRecv\\804复习1\\804复习1"
_dirlist = [path+"\\"+i for i in os.listdir(path)]    
_dirlist = [i+"\\"+os.listdir(i)[0] for i in _dirlist]
path2 = "E:\\qq_document\\2181778692\\FileRecv\\804复习1\\副本"
_dir2list = [path2+"\\"+i.split("\\")[-1].split(".")[0]+".docx" for i in _dirlist]

### 其他班

In [32]:
path="E:\\qq_document\\2181778692\\FileRecv\\806"
_dirlist = [path+"\\"+i for i in os.listdir(path) if i!="副本"]    
path2 = "E:\\qq_document\\2181778692\\FileRecv\\806\\副本"
_dir2list = [path2+"\\"+i.split("\\")[-1].split(".")[0]+".docx" for i in _dirlist]

### doc transform to docx

In [35]:
word = wc.Dispatch('Word.Application')

dirlist = zip(_dirlist,_dir2list)
for i in dirlist:
    DocSavesDocx(i[0],i[1])
word.Quit()

### get all answers

In [36]:
_dic = {}
for i in _dir2list:
    GetAnswer(i)

=== 807 陈思彻 ====>匹配失败!!!! 0
=== 809班 崔逸舟 ====>匹配失败!!!! 0
=== 刘佳灵-807班 ====>匹配失败!!!! 0


### merge all answer to a  table

In [37]:
import pandas as pd
a = 'BDDBCACDDDDDCDDDDCDABDBBDBCCCBDBCBDAADBBDDCDCDCBCBBDDDDBDBACDBC'
b = [i for i in a]
S = pd.Series(index=[i for i in range(1,64)],data=b,name="start")
df = pd.DataFrame(S)
df['id']=range(1,64)

for i in _dic.keys():
#     df[i]=[t[1] for t in _dic[i]]
    ansdict = {}
    ansdict = {"id":[int(j[0]) for j in _dic[i]],i:[j[1].upper() for j in _dic[i]]}

    _df = pd.DataFrame(ansdict)
    df = pd.merge(df,_df,on='id',how='left')

### 计算每个选项的准确性

In [38]:
for i in _dic.keys():
    df[i+"ans"] = df[["start",i]].apply(lambda x : 1 if x["start"]==x[i] else 0,axis=1)
df["accuracy"]= df[[i+"ans" for i in _dic.keys()]].apply(lambda x:sum(x)/35,axis=1)
_ansdic={}
for i in _dic.keys():
    _ansdic[i]=round(sum(df[i+"ans"])/63*100,1)
ss=pd.Series(data=[_ansdic[i] for i in _ansdic.keys()],index=[i for i in _ansdic.keys()],name="得分")

### 写入excel

In [39]:
df.rename(columns={"start":"标准答案"},inplace=True)
# writer = pd.ExcelWriter('C:\\Users\\shengyuan\\Desktop\\python\\805.xlsx')
# writer = pd.ExcelWriter('C:\\Users\\shengyuan\\Desktop\\python\\804.xlsx')
writer = pd.ExcelWriter('C:\\Users\\shengyuan\\Desktop\\python\\other.xlsx')
df.to_excel(writer,'学生情况',index=0)
ss.to_excel(writer,'得分情况')
writer.save()